# HW3 - Q3 [35 pts]

## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: <strong>REMOVE</strong> any print statements added to cells with "#export" that are used for debugging purposes befrore submitting because they will crash the autograder in Gradescope. Any additional cells can be used for testing purposes at the bottom. 
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove any comment that says "#export" because that will crash the autograder in Gradescope. We use this comment to export your code in these cells for grading.
</div>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> import any additional libraries into this workbook.
</div>

All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  

<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()` - 1 point

`trip_statistics()` - 3 points

`busiest_hour()` - 5 points

`most_freq_pickup_locations()` - 5 points

`avg_trip_distance_and_duration()` - 6 points

`most_freq_peak_hour_fares()` - 10 points

Each function will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from the NYC-TLC dataset during auto-grading.

In addition, you will also submit the resulting output csv from most_freq_peak_hour_fares() as output_large.csv.

`output_large.csv` - 5 points

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remove or modify the following utility functions:
</div>

`load_data()`

`main()`

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [1]:
#export
from pyspark.sql.functions import *
from pyspark.sql import *

Calculation started (calculation_id=b0cacb63-1e5c-755a-bd77-c4567888cc9e) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [2]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####

def load_data(size='small'):
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    
    if size != 'small' and size != 'large':
        print("Invalid size parameter provided. Use only 'small' or 'large'.")
        return
    
    input_bucket = "s3://cse6242-hw3-q3"
    
    # Load Trip Data
    trips_path = '/'+size+'/yellow_tripdata*'
    trips = spark.read.csv(input_bucket + trips_path, header=True, inferSchema=True)
    
    # Load Zone Data
    zones_path = '/'+size+'/taxi*'
    zones = spark.read.csv(input_bucket + zones_path, header=True, inferSchema=True)
    
    return trips, zones
    
def main(size, bucket):
    # Runs your functions
    trips, zones = load_data(size=size)
    
    print("User:", user())
    print()
    
    print("Trip Statistics:")
    ts = trip_statistics(trips)
    ts.show()
    print()
    
    print("Busiest Hour:")
    bh = busiest_hour(trips)
    bh.show(24)
    print()
    
    print("Most Frequent Pickup Locations:")
    mfpl = most_freq_pickup_locations(trips)
    mfpl.show()
    print()
    
    print("Average Trip Distance and Duration:")
    atdd = avg_trip_distance_and_duration(trips)
    atdd.show(n=24)
    print()
    
    print("Most Frequent Peak Hour Fares:")
    mfphf = most_freq_peak_hour_fares(trips, zones)
    mfphf.show()
    mfphf.coalesce(1).write.option("header","true").mode("overwrite").csv('{}/output_{}'.format(bucket, size))

Calculation started (calculation_id=fccacb63-29c4-511b-d1c8-b6db159570be) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


# Implement the below functions for this assignment:
<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> change any function inputs or outputs, and ensure that the dataframes your code returns align with the schema definitions commented in each function. Do <strong>NOT</strong> remove the #export comment from each of the code blocks either. This can prevent your code from being converted to a python file.
</div>

## 3.1 [1 pt] Update the `user()` function
This function should return your GT username, eg: gburdell3

In [3]:
#export
def user():
    return 'rpeng37'

Calculation started (calculation_id=7ecacb63-34f5-9176-4c7d-0b45af0e63a9) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.2 [3 pts] Update the `trip_statistics()` function
This function performs exploratory data analysis on the column trip_distance. Compute basic statistics (count, mean, stdev, min, max) for trip_distance. 

Example output formatting:

```
+-------+------------------+
|summary|     trip_distance|
+-------+------------------+
|  count|           xxxxxxx|
|   mean|           xxxxxxx|
| stddev|           xxxxxxx|
|    min|           xxxxxxx|
|    max|           xxxxxxx|
+-------+------------------+
```
Tip: Is there a PySpark Dataframe function you can use to solve this in a single line?

In [4]:
#export
def trip_statistics(trips):
    return trips.select("trip_distance").summary("count", "mean", "stddev", "min", "max")

Calculation started (calculation_id=5acacb63-7be4-020c-a57b-405414a8894e) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.3 [5 pts] Update the `busiest_hour()` function

Determine the hour of the day with the highest number of trips. Display the hour (0-23) and the corresponding trip count. 

Returns a PySpark DataFrame with a single row showing the hour with the highest trip count and the corresponding number of trips. Schema (hour, trip_count) 

Example output formatting:

```
+----+----------+
|hour|trip_count|
+----+----------+
|  xx|    xxxxxx|
+----+----------+
```

In [12]:
#export
from pyspark.sql import functions as F

def busiest_hour(trips):
    df = trips.withColumn("hour", hour("tpep_pickup_datetime"))
    result = df.groupBy("hour").agg(count("*").alias("trip_count")) \
               .orderBy(F.desc("trip_count")) \
               .limit(1)
    return result

Calculation started (calculation_id=a6cacb68-d34a-29ca-1727-7884a3e7e4aa) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.4 [5 pts] Update the `most_freq_pickup_locations()` function
Top 10 Most Frequent Pickup Locations

Identify the top 10 pickup locations (by PULocationID) with the highest number of trips. Display the location IDs along with their corresponding trip counts.

Example output formatting:
```
+------------+----------+
|PULocationID|trip_count|
+------------+----------+
|         xxx|    xxxxxx|
|         xxx|    xxxxxx|
|         xxx|    xxxxxx|
|         xxx|    xxxxxx|
|         ...|    ......|
+------------+----------+
```

In [6]:
#export
def most_freq_pickup_locations(trips): 
    result = trips.groupBy("PULocationID").agg(count("*").alias("trip_count")) \
                .orderBy(F.desc("trip_count")) \
                .limit(10)
    return result

Calculation started (calculation_id=6acacb64-1cad-e5f1-166e-eb55d008bf60) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.5 [6 pts] Update the `avg_trip_distance_and_duration()` function
Average Trip Distance and Duration by Hour

Calculate the average trip distance and average trip duration in minutes (i.e., divided by 60) for each hour of the day (0-23). Display the hour along with the corresponding averages. To be a valid trip, it must have a non-null timestamp and a trip distance greater than zero.

Note: You can use `unix_timestamp` to help with calculating the duration. If there are null or invalid timestamps, you will want to handle those accordingly. 

Expected Output:

A table with 24 rows showing each hour (0-23) along with the average trip distance and average trip duration for that hour.

Example output formatting:
```
+----+------------------+------------------+
|hour| avg_trip_distance| avg_trip_duration|
+----+------------------+------------------+
|   0|           xxxxxxx|           xxxxxxx|
|   1|           xxxxxxx|           xxxxxxx|
|   2|           xxxxxxx|           xxxxxxx|
|   3|           xxxxxxx|           xxxxxxx|
| ...|               ...|               ...|
|  23|           xxxxxxx|           xxxxxxx|
+----+------------------+------------------+
```

In [21]:
#export
def avg_trip_distance_and_duration(trips):
    valid = trips.filter((F.col("tpep_pickup_datetime").isNotNull()) & (F.col("trip_distance") > 0))
    valid = valid.withColumn("hour", hour("tpep_pickup_datetime"))
    valid = valid.withColumn("duration_minutes", (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime"))/60)
    result = valid.groupBy("hour") \
                  .agg(avg("trip_distance").alias("avg_trip_distance"),
                       avg("duration_minutes").alias("avg_trip_duration")) \
                  .orderBy("hour")
    return result

Calculation started (calculation_id=20cacb6d-8675-1198-2367-d361366384f6) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


## 3.6 [10 pts] Update the `most_freq_peak_hour_fares()` function
Top 10 Most Frequent Routes During Peak Hour 

Identify the top 10 most frequent routes (combinations of PULocationID and DOLocationID) during peak hours (7 AM - 9 AM and 4 PM - 7 PM). Peak hours can be defined as 7 <= hour < 9 and 16 <= hour < 19. Display the pickup and drop-off location ID pairs along with their trip counts and average total fare rounded to two decimal places. 

Note: A route must have a different drop off location from pickup location to be considered a valid route.

Expected Output:

A table showing the top 10 routes during peak hours with their trip counts and average total fare rounded to two decimal places. Each route is represented as a combination of PULocationID-DOLocationID and PULocationID should not be the same as DOLocationID.

Example output formatting:
```
+------------+------+------------+------+----------+--------------+
|PULocationID|PUZone|DOLocationID|DOZone|trip_count|avg_total_fare|
+------------+------+------------+------+----------+--------------+
|xxx         |xxx   |xxx         |xxx   |xxx       |xx.xx         |
|xxx         |xxx   |xxx         |xxx   |xxx       |xx.xx         |
|xxx         |xxx   |xxx         |xxx   |xxx       |xx.xx         |
|...         |...   |...         |...   |...       |...           |
+------------+------+------------+------+----------+--------------|

```

In [29]:
#export
from pyspark.sql import functions as F

def most_freq_peak_hour_fares(trips, zones):
    trips_with_hour = trips.withColumn("hour", F.hour("tpep_pickup_datetime"))
    trips_peak = trips_with_hour.filter(
        (F.col("PULocationID") != F.col("DOLocationID")) &
        (
            ((F.col("hour") >= 7) & (F.col("hour") < 9)) |
            ((F.col("hour") >= 16) & (F.col("hour") < 19))
        )
    )
    grouped = trips_peak.groupBy("PULocationID", "DOLocationID") \
               .agg(
                   F.count("*").alias("trip_count"),
                   F.avg("total_amount").alias("avg_total_fare")
               )
    grouped = grouped.withColumn("avg_total_fare", F.round("avg_total_fare", 2))
    top_routes = grouped.orderBy(F.desc("trip_count")).limit(10)
    zones_pickup = zones.withColumnRenamed("LocationID", "PU_ID") \
                         .withColumnRenamed("Zone", "PUZone")
    top_routes = top_routes.join(zones_pickup, top_routes.PULocationID == F.col("PU_ID"), "left").drop("PU_ID")
    zones_dropoff = zones.withColumnRenamed("LocationID", "DO_ID") \
                          .withColumnRenamed("Zone", "DOZone")
    top_routes = top_routes.join(zones_dropoff, top_routes.DOLocationID == F.col("DO_ID"), "left").drop("DO_ID")
    result = top_routes.select("PULocationID", "PUZone", "DOLocationID", "DOZone", "trip_count", "avg_total_fare")
    
    return result

Calculation started (calculation_id=f0cacb76-a6b0-fdfc-4bfc-7a8ef1e7e756) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


<div class="alert alert-block alert-info">
Once you have finished coding, you can export the notebook from `Notebook Explorer` by selecting your notebook and clicking `Export File` from the Actions dropdown.
</div>

#### Testing

<div class="alert alert-block alert-info">
    You may use the below cell for any additional testing you need to do, however any code implemented below will not be ran or used when grading. You can run the main function over the different sized datasets for testing your functions or you run them individually like in the examples below. To get the final output csv, you will need to run most_freq_peak_hour_fares(trips, zones) using the large dataset and write the resulting dataframe to a csv. The main function will do this for you, or you can do it yourself.
</div>

In [9]:
trips, zones = load_data('small')

Calculation started (calculation_id=6acacb66-3a83-9291-64a0-db9d22096ecd) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.


In [31]:
# ts = most_freq_peak_hour_fares(trips)
ts = most_freq_peak_hour_fares(trips, zones)
ts.show()

Calculation started (calculation_id=fccacb7a-1006-c5e0-f2d7-a564652bac29) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
+------------+--------------------+------------+--------------------+----------+--------------+
|PULocationID|              PUZone|DOLocationID|              DOZone|trip_count|avg_total_fare|
+------------+--------------------+------------+--------------------+----------+--------------+
|         237|Upper East Side S...|         236|Upper East Side N...|     16436|          8.68|
|         236|Upper East Side N...|         237|Upper East Side S...|     13196|          9.26|
|         239|Upper West Side S...|         238|Upper West Side N...|      7962|          7.77|
|         238|Upper West Side N...|         239|Upper West Side S...|      7319|          7.73|
|         239|Upper West Side S...|         142| Lincoln Square East|      7062|          8.22|
|         237|Upper East Side S...|         162|        Midtown East|      6651|          9.18|
|         142| Lincoln Square East|         239|Upper West Side S...|      6609|          8.38|
|         141|   

In [32]:
main('large', 's3://cse6242-rpeng37')

Calculation started (calculation_id=80cacb7a-3a2b-a46a-ae90-23fa3347dca5) in (session=40cacb62-efb9-2055-f295-b7e47950c81c). Checking calculation status...


Progress:   0%|          |elapsed time = 00:00s

Calculation completed.
User: rpeng37

Trip Statistics:
+-------+------------------+
|summary|     trip_distance|
+-------+------------------+
|  count|         187203269|
|   mean| 2.964925180927189|
| stddev|15.179487744870528|
|    min|              -.01|
|    max|             99.95|
+-------+------------------+


Busiest Hour:
+----+----------+
|hour|trip_count|
+----+----------+
|  18|  12112144|
+----+----------+


Most Frequent Pickup Locations:
+------------+----------+
|PULocationID|trip_count|
+------------+----------+
|         237|   7899292|
|         161|   7401958|
|         236|   7153325|
|         162|   6655847|
|         186|   6518420|
|         230|   6386265|
|          48|   5944600|
|         170|   5915604|
|         234|   5742552|
|         142|   5609771|
+------------+----------+


Average Trip Distance and Duration:
+----+------------------+------------------+
|hour| avg_trip_distance| avg_trip_duration|
+----+------------------+------------------+
|   0| 